# Input: PROFIL: Reduction de dimmension    
# Modele: TUNING Hyperparmètre d'un modele type LeNet 1D

In [3]:
import json
import random

import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Model

from contextlib import redirect_stdout
import keras_tuner as kt

from sklearn import preprocessing, model_selection
from sklearn import metrics


from pickle import dump, load

# Data preparation

In [4]:
# Acquisition
df_train = pd.read_json('Train_Intensite_H.json')
df_train['Intensite'] = df_train['Intensite'].apply(lambda x: np.array(x)/(np.array(x).max())).tolist()
df_test = pd.read_json('Test_Intensite_H.json')
df_test['Intensite'] = df_test['Intensite'].apply(lambda x: np.array(x)/(np.array(x).max())).tolist()

#Preprocessing
scaler = preprocessing.MinMaxScaler()
x_train_list = [ scaler.fit_transform(np.asarray(i).reshape(-1, 1)) for i in df_train['Intensite']]
y_train_list = [i for i in df_train['Diagnostic']]
x_test_list = [ scaler.fit_transform(np.asarray(i).reshape(-1, 1)) for i in df_test['Intensite']]
y_test_list = [i for i in df_test['Diagnostic']]

#Création des jeu d'entrainement, de validation et de Test
x_train, x_val, y_train, y_val = model_selection.train_test_split(x_train_list, y_train_list, test_size=0.2, random_state=42, shuffle=True)
x_train = np.asarray(x_train).astype(np.float32).reshape(-1, 256, 1)
y_train = np.asarray(y_train).astype(np.float32).reshape(-1, 1)
x_val = np.asarray(x_val).astype(np.float32).reshape(-1, 256, 1)
y_val = np.asarray(y_val).astype(np.float32).reshape(-1, 1)
x_test = np.asarray(x_test_list).astype(np.float32).reshape(-1, 256, 1)
y_test = np.asarray(y_test_list).astype(np.float32).reshape(-1, 1)

In [5]:
def plot_history_metrics(history):
 acc = history.history['accuracy']
 val_acc = history.history['val_accuracy']

 loss = history.history['loss']
 val_loss = history.history['val_loss']

    
 fig = plt.figure(figsize=(12,6))
 ax1 = fig.add_subplot(121)

 ax1.plot(acc, label='Training Accuracy')
 ax1.plot(val_acc, label='Validation Accuracy')
 ax1.legend(loc='lower right')
 ax1.set_title('Training and Validation Accuracy')

 ax2 = fig.add_subplot(122)
 ax2.plot(loss, label='Training Loss')
 ax2.plot(val_loss, label='Validation Loss')
 ax2.legend(loc='upper right')
 ax2.set_title('Training and Validation Loss')
 plt.show()
    
 fig.savefig("Training and Validation.png")
 plt.close(fig)
    

# SET UP

### Define simple function to plot all the metrics present in a keras.callbacks.History

### build_model function to generate 1D LeNet Like

In [10]:
def build_model():

    # Model Variables
    loss = keras.losses.BinaryCrossentropy()
    thresh = 0.3
    l_rate = 0.00011181404254174737
    adam = tf.keras.optimizers.Adam(learning_rate= l_rate)
    metric = [keras.metrics.BinaryAccuracy(name ='accuracy', threshold= thresh)]


    # Model architecture    
    input_layer = keras.Input(shape=(256, 1))

    x = layers.Conv1D(filters=256,kernel_size=3, strides=2, activation="relu")(input_layer)
    x = layers.Conv1D(filters=512,kernel_size=5, strides=2, activation="relu")(x)
    x = layers.Conv1D(filters=256,kernel_size=5, strides=2, activation="relu")(x)
    x = layers.Conv1D(filters=1024,kernel_size=5, strides=2, activation="tanh")(x)
    x = layers.Conv1D(filters=128,kernel_size=5, strides=2, activation="relu")(x)
    x = layers.Conv1D(filters=32,kernel_size=5, strides=2, activation="relu")(x)
    
    
    x = layers.BatchNormalization()(x)
    x = layers.Flatten()(x)

    x = layers.Dense(256, activation="relu", kernel_regularizer=keras.regularizers.L2())(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(units=64, activation="relu", kernel_regularizer=keras.regularizers.L2())(x)
    x = layers.Dense(units=512, activation="relu", kernel_regularizer=keras.regularizers.L2())(x)
    x = layers.Dense(units=416, activation="relu", kernel_regularizer=keras.regularizers.L2())(x)
    x = layers.Dense(units=288, activation="tanh", kernel_regularizer=keras.regularizers.L2())(x)
    x = layers.Dense(units=224, activation="tanh", kernel_regularizer=keras.regularizers.L2())(x)
    
    
    output_layer = layers.Dense(1, activation="relu")(x)

    model = Model(inputs=input_layer, outputs=output_layer)

    # Modele compilation
    model.compile(optimizer=adam, loss=loss, metrics=metric)
    
    
    return model 



### Instanciate main variables: callbacks, optimizer, loss , metrics, epoch

In [12]:
PROJECT_NAME = "1D-LeNet-like Tuning"

callbacks = [
    keras.callbacks.ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_accuracy"),
    keras.callbacks.ReduceLROnPlateau( monitor="val_accuracy", factor=0.2, patience=10, min_lr=0.000001),
    keras.callbacks.EarlyStopping( monitor="val_accuracy", patience=8, verbose=1, mode="auto", restore_best_weights=True, start_from_epoch=10)]
 

In [9]:
# Importer MLflow
import mlflow
import mlflow.keras

try:
    experiment_id = mlflow.create_experiment('Input Profil  ouput: Binary Malade/Normal')
except:
    experiment_name = 'Input Profil  ouput: Binary Malade/Normal'
    current_experiment=dict(mlflow.get_experiment_by_name(experiment_name))
    experiment_id=current_experiment['experiment_id']



with mlflow.start_run(experiment_id =experiment_id) as run:
    model_uri = f"runs:/{run.info.run_id}"
    mv = mlflow.register_model(model_uri, "Best 1D Le Net-like ")
    
    hypermodel = build_model()
    keras.utils.plot_model(hypermodel,to_file="Model.png", show_shapes=True)

    mlflow.log_param("seuil", 0.3)
    mlflow.log_param("loss", 'BinaryCrossentropy')
    mlflow.log_param("learning rate", 0.00011181404254174737)
    mlflow.log_param("epoch", 126)
 
    
    history = hypermodel.fit(x_train, y_train, epochs=126, validation_data=(x_val, y_val))

    eval_result = hypermodel.evaluate(x_test, y_test)
    y_pred = hypermodel.predict(x_test)
    y_pred_class = np.apply_along_axis(lambda x: 1 if x>0.3 else 0, 1, y_pred)

    confusion_m = metrics.confusion_matrix(y_test, y_pred_class)
                
    VN = confusion_m[0,0]
    VP = confusion_m[1,1]
    FN = confusion_m[1,0]
    FP = confusion_m[0,1]

    plot_history_metrics(history)
    print('Justesse du jeu de test =', np.round((VP+VN)/(VP+VN+FP+FN)*100,2),'%')
    print('Sensibilité =', np.round(VP*100/(VP+FN),2),'%')
    print('Spécificité =', np.round(VN*100/(VN+FP),2),'%')

    print('---------Matrice de confusition-----------------')
    print('               pred:Malade     pred:Normal')
    print('vrai Malade:      ',VP,'            ',FN)
    print('vrai Normal       ',FP,'            ',VN)
        
    signature = mlflow.models.infer_signature(x_train, hypermodel.predict(x_train))
    mlflow.keras.log_model(hypermodel, artifact_path="model", signature=signature)
    
    mlflow.log_metric("Justesse",(VP+VN)/(VP+VN+FP+FN))
    mlflow.log_metric("VN", VN)
    mlflow.log_metric("VP", VP)
    mlflow.log_metric("FN", FN)
    mlflow.log_metric("FP", FP)
    mlflow.log_metric("Sensibilite", VP/(VP+FN))
    mlflow.log_metric("Specificite", VN/(VN+FP))
    mlflow.log_param("seuil", 0.3)

    mlflow.log_artifact("Training and Validation.png")
    mlflow.log_artifact("Model.png")
    mlflow.end_run()

with open('./Model/last_exp3-1.pkl', 'wb') as pickle_file:
    dump(all, pickle_file)
    
tf.keras.saving.save_model(hypermodel, './Model/last_exp3-1.keras')          

Registered model 'Best 1D Le Net-like ' already exists. Creating a new version of this model...
Created version '6' of model 'Best 1D Le Net-like '.


ValueError: Expected `metrics` argument to be a list, tuple, or dict. Received instead: metrics=<keras.src.metrics.accuracy_metrics.BinaryAccuracy object at 0x132722fd0> of type <class 'keras.src.metrics.accuracy_metrics.BinaryAccuracy'>

In [ ]:
help(mlflow.models.infer_signature)